In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=12
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_096.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_094', 'Input_C_099', 'Input_C_098', 'Input_C_044',
       'Input_A2_014', 'Input_C_137', 'Input_C_121', 'Input_C_056',
       'Input_C_101', 'Input_A4_019', 'Input_C_031_Y', 'Input_C_093'],
      dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,12)

train_rmse: 0.006143586064844172
test_rmse: 0.0063764056586489825
test_r2: 0.1687728929586918


## Lasso

In [6]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.006292975925823191
[0.006338504676476967, 0.006062411531315351, 0.006756448502576744, 0.006159655198779472, 0.006777498442074015, 0.005943017246401346, 0.006384401133030058, 0.0058201665761850995, 0.006100849185901555, 0.0065868067654912986]


test_r2_ave: 0.16096551814725918
[0.09259492247607826, 0.1899062932838167, 0.07120628928242556, 0.27102995290097476, 0.026295263958347692, 0.26312041870611547, 0.10653263795386536, 0.2186253495261854, 0.22886798208450976, 0.14147607130027273]


## ElasticNet

In [5]:
ElasticNet_rmse_ave(skew_data[title],y,[0.01],0.01)

test_rmse_ave: 0.006289273290400697
[0.006309869575024675, 0.006067455378125738, 0.0067317150049022376, 0.00623161448908959, 0.00675453300122084, 0.005966898073577268, 0.006331983949333754, 0.005779588843976547, 0.0061072339791669854, 0.006611840609589328]


test_r2_ave: 0.16241116069293454
[0.10077506759844945, 0.18855775787606954, 0.0779939586952878, 0.2538982884239481, 0.03288284869335634, 0.2571865138211631, 0.1211434957756572, 0.22948273088664783, 0.2272530938293067, 0.13493785132945957]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,12,model)

train_rmse: 0.006294405985045472
test_rmse: 0.006587524105666151
test_r2: 0.1105445248595451


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 80, 
              max_depth = 7, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 0.07)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.006282132519222832
[0.0060121788839655205, 0.006509839485702635, 0.006495373181560368, 0.006128907271766247, 0.0065520018274168295, 0.005878765952845115, 0.0062647187896699924, 0.006416138937363722, 0.0059280377216234725, 0.0066353631403144216]


test_r2_ave: 0.16292394060992094
[0.18362186823972948, 0.06591802672644376, 0.14159835791803443, 0.278289571465401, 0.09001035728779927, 0.2789674314637006, 0.1397166394730185, 0.050410272144453705, 0.2719351336761304, 0.12877174770449806]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 3,
            max_leaf_nodes = 5,
            min_samples_leaf =6,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.00627877481303907
[0.006270561187322562, 0.0063954843652325455, 0.0063529031646757375, 0.0065320570710004685, 0.006338191133403963, 0.005798670049022241, 0.006055455500053276, 0.006164245252022048, 0.006212951229945163, 0.006667229177712691]


test_r2_ave: 0.1656756548571546
[0.1119439139827555, 0.09844689497170656, 0.1788418734674554, 0.18022094041686376, 0.14843247176227892, 0.29848116380152157, 0.19622962618881012, 0.1235072923042122, 0.20026878775105006, 0.12038358392489201]


In [10]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-05)
elastic_mod=ElasticNet(alpha=[0.01], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 80, 
              max_depth = 7, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.2,
              reg_alpha =  0,
              reg_lambda = 0.07)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 9,
            min_samples_split = 3,
            max_leaf_nodes = 5,
            min_samples_leaf =6,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.006156191629535479
[0.006149540380146934, 0.006009519226042523, 0.006564827098164792, 0.006049785440844782, 0.006563922906411568, 0.0058093568871965895, 0.006146241074441159, 0.005882450374000101, 0.00593148827165034, 0.006454784636456008]


test_r2_ave: 0.1972801398717685
[0.145891802517639, 0.20398016739148228, 0.12314272675251736, 0.29680329954033136, 0.08669597262075535, 0.29589300939851615, 0.17194813092887729, 0.20181229719644156, 0.2710873133649685, 0.17554667900615617]


In [12]:

stack_mod = StackingRegressor(regressors=[lasso_mod, random_mod, elastic_mod, xgb_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.006220952616068357
[0.006008567366637697, 0.006346811897724723, 0.006665347234504997, 0.005842503735965991, 0.0066258941741886325, 0.005828561917982814, 0.006178431679574568, 0.006273360476500642, 0.005842255236216951, 0.0065977924413865505]


test_r2_ave: 0.17844855029172085
[0.18460237074452035, 0.11211711461508567, 0.0960843985691967, 0.3441645665995109, 0.0693692037153576, 0.2912299293373928, 0.16325166303282057, 0.09220251457402595, 0.2928538029898088, 0.1386099387394889]


In [13]:
ave(skew_data[title],y,0.2,stack_mod,0.6,vote_mod,0.2,random_mod)

test_rmse_ave: 0.00610997948029416
[0.006056370122582801, 0.006069028594389502, 0.006462786547782862, 0.006037726698992849, 0.006449529084522898, 0.005730238175099687, 0.006051834663286935, 0.0059301310253998905, 0.005860630774143436, 0.0064515191167407475]


test_r2_ave: 0.20934902477042536
[0.1715765353281855, 0.18813690941397065, 0.15018978192286714, 0.2996038009833266, 0.11825208643347995, 0.31494113822969527, 0.19719056173307759, 0.18882031302707492, 0.2883984594853988, 0.17638066114717732]
